In [21]:
# the imports in this cell are required when running from Cloud (Colab/Kaggle)
# before running on cloud you nee to upload the .py files 
# from 'Notebooks/utils' directory
from applyML_util import train_regression, eval_regression, showEvalutationGraph_regression
from featureSelection_util import (pearson_correlation_fs, 
                                   seleckKBest_fs, selectSequential_fs)

# New Section

**LGBMR Documentation link:** https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html

**XGBoost Documentation link:** https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier

**SK-Lego Documentation link:** https://scikit-lego.netlify.app/meta.html

In [ ]:
!pip install sklego

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklego.meta import ZeroInflatedRegressor #!pip install sklego
from xgboost.sklearn import XGBClassifier
from lightgbm import LGBMRegressor

In [ ]:
# global random seed
RAND_SEED = 42

ziReg = ZeroInflatedRegressor(classifier=XGBClassifier(random_state=RAND_SEED,tree_method='hist'), regressor=LGBMRegressor(random_state=RAND_SEED))

# initial model with only random seed and not any hyper-parametes
initial_model = ziReg

# hyper-parameters
#LGBM
n_estimators = [x*5 for x in range(20, 41)] 
learning_rate = [0.01]
#XGB
subsample = [0.5]

param_grid = { 'classifier__subsample' :subsample,
              'classifier__n_estimators': n_estimators,
              'regressor__learning_rate': learning_rate,
              'regressor__n_estimators': n_estimators
              }

# variables needed for showEvalGraph_regression() function
# MODEL_CLASS = ziReg
# x_axis_param_name = 'regressor__C'
# x_axis_vals = regressor__C

## 1. Experimentation on the Weather Daily dataset

In [ ]:
# Load the train dataset
weather_daily_train_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/train/brri-weather_train_regression.csv')

# Load the test set
weather_daily_test_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/test/brri-weather_test_regression.csv')

In [ ]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__n_estimators': 175, 'classifier__subsample': 0.5, 'regressor__learning_rate': 0.01, 'regressor__n_estimators': 175}
Train set performance: r2-score=0.2506, mae=5.519, rmse=12.9369


In [ ]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [ ]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1886, mae=5.7885, rmse=15.5752


### 1.1 Apply Pearson Feature Selection to Daily Weather Dataset

In [ ]:
# select features from the train dataset
weather_daily_fs1_train_df, cols_to_drop = pearson_correlation_fs(weather_daily_train_df, 'Rainfall (mm)')

# keep only selected features on the test dataset
weather_daily_fs1_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

dropping Sunshine (hour/day) from (Cloudy (hour/day), Sunshine (hour/day))


In [ ]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs1_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__n_estimators': 175, 'classifier__subsample': 0.5, 'regressor__learning_rate': 0.01, 'regressor__n_estimators': 200}
Train set performance: r2-score=0.2543, mae=5.5168, rmse=12.9129


In [ ]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs1_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [ ]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs1_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1854, mae=5.7485, rmse=15.6055


### 1.2 Apply SelectKBest Feature Selection to Daily Weather Dataset

In [ ]:
# select features from the train dataset
weather_daily_fs2_train_df, cols_to_drop = seleckKBest_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs2_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

features dropped: ['Max Temp. (degree Celcius)', 'Solar Radiation (cal/cm^2/day)']


In [ ]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs2_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__n_estimators': 180, 'classifier__subsample': 0.5, 'regressor__learning_rate': 0.01, 'regressor__n_estimators': 180}
Train set performance: r2-score=0.2403, mae=5.5091, rmse=13.0274


In [ ]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs2_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [23]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs2_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=-0.1305, mae=6.2186, rmse=18.3845


### 1.3 Apply SelectSequential Feature Selection to Daily Weather Dataset

In [ ]:
# select features from the train dataset
weather_daily_fs3_train_df, cols_to_drop = selectSequential_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs3_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

In [ ]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs3_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__n_estimators': 105, 'classifier__subsample': 0.5, 'regressor__learning_rate': 0.01, 'regressor__n_estimators': 160}
Train set performance: r2-score=0.2129, mae=5.6948, rmse=13.2398


In [ ]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs3_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [24]:
# # test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs3_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1152, mae=6.1534, rmse=16.2643
